In [20]:
import nidaqmx
from nidaqmx.constants import AcquisitionType, RegenerationMode
import numpy as np
import time

# Settings
channel = "myDAQ1/ao0"            # Change to your device/channel
sample_rate = 1000                # 1000 samples per second
batch_size = 100                  # Send 100 samples per batch (0.1s of data)
batch_time = batch_size/sample_rate
duration = 30                     # Total time to run in seconds

t = np.linspace(0, duration, duration * sample_rate)

signal = np.sin(t)

numBatches = len(signal) / batch_size

# Create a new task
with nidaqmx.Task() as task:
    # 1. Add an analog output voltage channel
    task.ao_channels.add_ao_voltage_chan(channel)

    # 2. Configure the sample clock for continuous output
    task.timing.cfg_samp_clk_timing(
        rate=sample_rate,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=batch_size * 10  # Buffer size (make it bigger than one batch)
    )

    # 3. Disable regeneration to force us to keep writing new data
    task.out_stream.regen_mode = RegenerationMode.DONT_ALLOW_REGENERATION
    start_time = time.time()
    initial_batch = signal[0:batch_size]
    task.write(initial_batch, auto_start=False)
    # 4. Start the task
    task.start()

    print("Outputting random signal for 30 seconds...")
    batch_number = 0
    while batch_number < numBatches-1:
        genStart = time.time()
        batch_number += 1
        print(batch_number)
        # 5. Generate a batch of random voltage values between -2V and 2V
        voltage_batch = signal[batch_number * batch_size: (batch_number + 1) * batch_size]

        time.sleep(batch_size / sample_rate - 1.1 * (time.time() - genStart))  # 0.1s in this case

        # 6. Write the batch to the device (do not auto-start; task already started)
        task.write(voltage_batch, auto_start=False)

        # 7. Wait a short time before sending the next batch (match sample rate)
        

    print("Done.")

Outputting random signal for 30 seconds...
1


DaqWriteError: The generation has stopped to prevent the regeneration of old samples. Your application was unable to write samples to the background buffer fast enough to prevent old samples from being regenerated.

To avoid this error, you can do any of the following:
1. Increase the size of the background buffer by configuring the buffer.
2. Increase the number of samples you write each time you invoke a write operation.
3. Write samples more often.
4. Reduce the sample rate.
5. If your data transfer method is interrupts, try using DMA or USB Bulk.
6. Reduce the number of applications your computer is executing concurrently.

In addition, if you do not need to write every sample that is generated, you can configure the regeneration mode to allow regeneration, and then use the Position and Offset attributes to write the desired samples.
Task Name: _unnamedTask<12>

Status Code: -200290